In [1]:
import numpy as np
import pandas as pd

In [42]:
mex_osm = pd.read_csv('tokyo_mex_start_end_nodes.csv')
group_measures = pd.read_csv('group_measure.csv')
group_measures = pd.merge(group_measures, mex_osm[['start', 'end', 'obs_grp_id']], how='left', left_on='obs_grp', right_on='obs_grp_id')
group_measures = group_measures[['obs_grp', 'dir', 'start', 'end', 'Ｑ']]
group_measures.head(20)

,obs_grp,dir,start,end,Ｑ
0,0,1,203569,139096,11473.50
1,1,1,139096,251226,7893.50
2,2,1,251226,19862,6475.00
3,3,2,230024,305949,4738.75
4,6,1,138546,11536,42489.67
5,7,2,368763,11535,35759.80
6,8,1,11536,339389,46855.17
7,10,1,339389,628883,63007.00
8,12,1,628883,628492,23232.00
9,14,1,628492,641537,61197.00


In [7]:
import scipy.sparse as ssparse
from ctypes import *
import scipy.io as sio

sys.path.insert(0, '/home/bingyu')
from sp import interface

In [9]:
links_df0 = pd.read_csv('../osm/tokyo_edges.csv')
links_df0['fft'] = links_df0['length']/links_df0['maxmph']*2.237
links_df0['fft'] = np.where(links_df0['type'].isin(['motorway', 'motorway_link']), links_df0['fft']*0.1, links_df0['fft'])
nodes_df0 = pd.read_csv('../osm/tokyo_nodes.csv')

wgh = links_df0['fft']
row = links_df0['start_igraph']
col = links_df0['end_igraph']
g_coo = ssparse.coo_matrix((wgh, (row, col)), shape=(nodes_df0.shape[0], nodes_df0.shape[0]))
sio.mmwrite('network_sparse_mex_analysis.mtx', g_coo)
g = interface.readgraph(bytes('network_sparse_mex_analysis.mtx', encoding='utf-8'))


In [76]:
links_df0['fft2'] = links_df0['fft']*1.2
links_df0[["edge_id_igraph", "start_igraph", "end_igraph", "fft2"]].to_csv('../osm/tokyo_edges_discount.csv', index=False)

In [100]:
### add geometry
quarter_edge_vol = pd.read_csv('../simulation_outputs/edge_vol_distance/small_edge_vol_h{}_q{}.csv'.format(5, 0))
print(quarter_edge_vol.shape)
quarter_edge_vol = pd.merge(quarter_edge_vol, links_df0[['edge_id_igraph', 'length', 'geometry']], how='left', left_on='edgeid', right_on='edge_id_igraph')
print(np.sum(quarter_edge_vol['length']*quarter_edge_vol['vol'])/19876)
# quarter_edge_vol.to_csv('../simulation_outputs/edge_vol_geom/edge_vol_discount_geom_h{}_q{}.csv'.format(5, 0))

(449159, 2)
9625.961086562576


In [56]:
demand_file = pd.read_csv('../osm/tokyo_demands_0.csv')
demand_file = pd.merge(demand_file, nodes_df0, how='left', left_on='node_id_igraph_O', right_on='node_id_igraph')
demand_file = pd.merge(demand_file, nodes_df0, how='left', left_on='node_id_igraph_D', right_on='node_id_igraph', suffixes=['_O', '_D'])
demand_file.head()

,agent_id,node_id_igraph_O,node_id_igraph_D,hour,node_id_igraph_O,node_osmid_O,lon_O,lat_O,signal_O,node_id_igraph_D,node_osmid_D,lon_D,lat_D,signal_D
0,0,671847,149554,10,671847,6161361393,139.551513,35.741041,NaN,149554,5302060801,139.538243,35.722554,NaN
1,1,420918,262011,20,420918,5351118925,139.497777,35.760877,NaN,262011,659312275,139.522262,35.690114,NaN
2,2,280657,60268,11,280657,3874301808,139.481140,35.900206,NaN,60268,6396504346,139.483163,35.892384,NaN
3,3,527117,247736,20,527117,1014530112,139.701100,35.657521,NaN,247736,309049161,139.674824,35.666698,NaN
4,4,20453,175092,7,20453,302055617,139.719520,35.680052,NaN,175092,1160264526,139.620114,35.691736,NaN


In [57]:
### trip distance straightline
import haversine
dist = haversine.haversine(demand_file['lon_O'], demand_file['lat_O'], demand_file['lon_D'], demand_file['lat_D'])
print(np.mean(dist))

7662.484986717428


In [85]:
obs_grp_edge_list = []
for row in group_measures.itertuples():
    obs_grp_id = getattr(row, 'obs_grp')
    dir = getattr(row, 'dir')
    if dir == 1:
        start_igraph = getattr(row, 'end')
        end_igraph = getattr(row, 'start')
    elif dir == 2:
        start_igraph = getattr(row, 'start')
        end_igraph = getattr(row, 'end')
    else:
        print('invalid direction')
    # print(start_igraph, end_igraph)

    sp = g.dijkstra(start_igraph+1, end_igraph+1)
    sp_dist = sp.distance(end_igraph+1)
    if sp_dist > 10e7:
        print('route not found')
        sp.clear()
    else:
        sp_route = sp.route(end_igraph+1)
        route_igraph = [(start_sp-1, end_sp-1) for (start_sp, end_sp) in sp_route]
        obs_grp_edge_list += [(obs_grp_id, start, end) for (start, end) in route_igraph]
        sp.clear()

obs_grp_edge_df = pd.DataFrame(obs_grp_edge_list, columns=['obs_grp_id', 'start_igraph', 'end_igraph'])
# print(obs_grp_edge_df.groupby('obs_grp_id').size())
obs_grp_edge_df = pd.merge(obs_grp_edge_df, links_df0[['start_igraph', 'end_igraph', 'edge_id_igraph', 'length']], how='left', on=['start_igraph', 'end_igraph'])
obs_grp_edge_df.head()

,obs_grp_id,start_igraph,end_igraph,edge_id_igraph,length
0,0,139096,206766,1458307,789.702933
1,0,206766,204000,7893,155.144367
2,0,204000,203569,1401172,144.315274
3,1,251226,314947,7895,72.258353
4,1,314947,139096,1458306,106.790187


In [104]:
daily_edge_vol = obs_grp_edge_df.copy()
daily_edge_vol = daily_edge_vol.drop_duplicates(subset='edge_id_igraph')
daily_edge_vol['daily_vol'] = 0
for hour in range(5,6):
    for quarter in range(1):
        quarter_edge_vol = pd.read_csv('../simulation_outputs/edge_vol/edge_vol_h{}_q{}.csv'.format(hour, quarter))
        quarter_edge_vol = quarter_edge_vol.loc[quarter_edge_vol['edgeid'].isin(daily_edge_vol['edge_id_igraph'])]
        daily_edge_vol = pd.merge(daily_edge_vol, quarter_edge_vol, how='left', left_on='edge_id_igraph', right_on='edgeid')
        daily_edge_vol['daily_vol'] += daily_edge_vol.fillna(value={'vol': 0})['vol']
        daily_edge_vol = daily_edge_vol[['obs_grp_id', 'edge_id_igraph', 'length', 'daily_vol']]
        # break
    # break
daily_edge_vol.sort_values(by='daily_vol', ascending=False).head()

,obs_grp_id,edge_id_igraph,length,daily_vol
298,61,211,1325.302875,215.0
297,61,1164751,60.669454,215.0
299,61,150,60.102157,215.0
248,57,1140632,847.686902,158.0
244,57,7823,52.256742,155.0


In [78]:
daily_edge_vol = daily_edge_vol.merge(links_df0[['edge_id_igraph', 'type']], how='left', on='edge_id_igraph')
daily_edge_vol.groupby('type').size()

type
motorway         363
motorway_link     26
primary            4
secondary          1
dtype: int64